   # CW ODMR with AWG5014C

   ## Load modules and set parameters

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [2]:
try:
    odmr
except NameError:
    manager.startModule('gui', 'odmr')
    
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')    

try: 
    odmrlogic
except NameError:
    manager.startModule('logic', 'odmrlogic')

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
    
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')


In [3]:
odmrlogic.mw_start = 1.498e+9
odmrlogic.mw_stop = 1.518e+9
odmrlogic.mw_step =0.5e+6
odmrlogic.run_time = 50.0
odmrlogic.sweep_mw_power = -25

odmrlogic.cw_mw_frequency = odmrlogic.mw_start
odmrlogic.cw_mw_power = odmrlogic.sweep_mw_power

odmr._mw.cw_frequency_DoubleSpinBox.setValue(odmrlogic.cw_mw_frequency)
odmr._mw.start_freq_DoubleSpinBox.setValue(odmrlogic.mw_start)
odmr._mw.stop_freq_DoubleSpinBox.setValue(odmrlogic.mw_stop)
odmr._mw.step_freq_DoubleSpinBox.setValue(odmrlogic.mw_step)
odmr._mw.cw_power_DoubleSpinBox.setValue(odmrlogic.cw_mw_power)
odmr._mw.sweep_power_DoubleSpinBox.setValue(odmrlogic.sweep_mw_power)
odmr._mw.runtime_DoubleSpinBox.setValue(odmrlogic.run_time)

In [4]:
 # load Laser_MW_On sequence to AWG
pulsedmasterlogic.clear_pulse_generator()
time.sleep(5)
sequencegeneratorlogic.generate_laser_mw_on(name='Laser_MW_On', laser_amp=2.0, mw_channel='a_ch1',
                         mw_freq=0.1e+9, mw_amp=0.25)
if 'Laser_MW_On' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
    sequencegeneratorlogic.log.error('Ensemble generation of Rabi in notebook timed out.')
pulsedmasterlogic.sample_block_ensemble('Laser_MW_On', True)
pulsedmasterlogic.load_asset_into_channels('Laser_MW_On')

In [6]:
def do_cw_odmr(save_data=False):
    
    # perform measurement
    odmrlogic.start_odmr_scan()
    
    time.sleep(odmrlogic.run_time+0.5)
    
    odmrlogic.do_fit('Lorentzian dip')
    
    if save_data:
        odmr.save_data()
        
    pulsedmeasurement.laser_on()
    
    print('finish')

In [7]:
do_cw_odmr()

finish
